#Goal: Understand the concept behind .backward() method
Synopsis:
Gradient will be calculated in 2 ways and confirm if they match:
(i) Numerical gradient (Calculus- approximation of differentation operator)
(ii) Analytical gradient (via Autograd- method used by PyTorch)
Note: Catastrophic cancellation - Digits beyond 7 SF in 32-bit floats are prone to noise and are unreliable. The numerical gradient method is susceptible to this whereas analytical gradient (Autograd) is not because subtracting two nearly identical loss values cancels meaningful digits, and dividing by a tiny epsilon amplifies the remaining noise.
This is a core motivation for autograd — it computes exact analytical gradients regardless of scale.

In [ ]:
import torch
torch.manual_seed(42)
x = torch.randn(50) # one descriptor across 50 molecules
y_true = 0.7 * x + 0.1

w = torch.tensor(1.5, requires_grad=True) # weight

# Analytical gradient via autograd
y_pred = w * x
loss = ((y_pred - y_true) ** 2).mean()
loss.backward()
analytical_gradient = w.grad.item()

# Numerical gradient 
epsilon = 1e-4
with torch.no_grad():
    loss_plus = (((w + epsilon) * x - y_true) ** 2).mean()
    loss_minus = (((w - epsilon) * x - y_true) ** 2).mean()
numerical_gradient = (loss_plus - loss_minus) / (2 * epsilon)

print(f"Analytical gradient: {analytical_gradient:.6f}")
print(f"Numerical gradient:  {numerical_gradient:.6f}")
print(f"Difference:          {abs(analytical_gradient - numerical_gradient):.2e}")    


Analytical gradient: 1.720351
Numerical gradient:  1.720786
Difference:          4.35e-04


In [12]:
with torch.no_grad():
    for eps in [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-7]:
        loss_plus  = (((w + eps) * x - y_true) ** 2).mean()
        loss_minus = (((w - eps) * x - y_true) ** 2).mean()
        num_grad = ((loss_plus - loss_minus) / (2 * eps)).item()
        print(f"eps={eps:.0e}  diff={abs(analytical_gradient - num_grad):.2e}")
        print((loss_plus - loss_minus).item())

eps=1e-01  diff=9.54e-07
0.3440704345703125
eps=1e-02  diff=3.22e-06
0.034406960010528564
eps=1e-03  diff=1.07e-04
0.0034409165382385254
eps=1e-04  diff=4.35e-04
0.00034415721893310547
eps=1e-05  diff=2.22e-03
3.445148468017578e-05
eps=1e-07  diff=3.66e-01
4.172325134277344e-07
